In [1]:
import win32com.client
from win32com.client import constants as c
import pandas as pd
import datetime

now = str(datetime.datetime.now())[:19]                        # Generate a time stamp when the script is started
now = now.replace(":","_")


In [2]:
################################################## Helper Functions ########################################################

def QuitApp (PPTApp, Open_presentations_list):
    
    print("Status: Dispatched PowerPoint Object Killed.")
    if Open_presentations_list == []:
        PPTApp.Quit()

def SavePPT (PPTApp, PPTPres, Out_File_Name):
    
    PPTPres.SaveAs(Out_File_Name)

def ClosePPT (PPTApp, PPTPres, Open_presentations_list):
    
    print("Status: Closing Presentation ...")
    if PPTPres.FullName not in Open_presentations_list:
        PPTApp.Presentations(PPTPres.FullName).Close()

In [3]:
Input_Excel = r"C:/.../InputData.xlsx"  ## Excel having Charts
Input_PPT = r'C:/.../Template_Bar.pptx' ## Template
Output_Result = r"C:/../Presentation_Result_Auto" + str(now) + ".pptx"

In [4]:
def AutomatePPT():

    # Picks an Active Instance of Excel.
    Excel = win32com.client.GetActiveObject("Excel.Application")
    Excel.Visible = True

    # The workbook with the charts.
    xlWorkbook = Excel.Workbooks.Open(Input_Excel)
    
    # The metadata sheet which contains all the information about the data to be placed in PPT
    df = pd.read_excel(Input_Excel,sheet_name = 'MetaData')

    try: 

        PPTApp = win32com.client.GetActiveObject("Powerpoint.Application")
        PPTApp.Visible = True
        Open_presentations_list = []
        for PPTs in PPTApp.Presentations:
            Open_presentations_list.append(PPTs.FullName)

        PPTPres = PPTApp.Presentations(Input_PPT)   ### If App Aleready Open

    except:

        PPTApp = win32com.client.Dispatch("PowerPoint.Application")
        PPTApp.Visible = True
        Open_presentations_list = []
        for PPTs in PPTApp.Presentations:
            Open_presentations_list.append(PPTs.FullName)

        PPTPres = PPTApp.Presentations.Open(Input_PPT, WithWindow = False)

    #Add a presentation to the PowerPoint Application, returns a Presentation Object.
    PPTPresentation = PPTPres #PPTApp.Presentations.Add()

    ind_sheet = 1

    for i in range(df.shape[0]):

        # Wee need to check each Worksheet as mentioned in the Metadata Sheet.
        for xlWorksheet in xlWorkbook.Worksheets:

            if xlWorksheet.Name == df['Sheet'][i]:

                ind_chart = 1

                # Grab the ChartObjects Collection for each sheet.
                xlCharts = xlWorksheet.ChartObjects()

                # Loop through each Chart in the ChartObjects Collection.

                for index, xlChart in enumerate(xlCharts):

                    if index+1 == int(df['Chart'][i]):

                        PPTSlide = PPTPresentation.Slides(int(df['Slides'][i]))  # 12 is a blank layout

                        # Display something to the user.
                        print('Exporting Chart {} from Worksheet {}'.format(xlChart.Name, xlWorksheet.Name))

                        # Copy chart.
                        xlChart.Copy()

                        # Paste the Object to the Slide
                        PPTSlide.Shapes.PasteSpecial(DataType = 1)
                        Shapes_Count_New = PPTSlide.Shapes.Count

                       # after puting the chart in the slides where they are meant to be, shape them according to the values provided in Metadata sheet

                        PPTSlide.Shapes(Shapes_Count_New).Width = int(df['Width'][i])
                        PPTSlide.Shapes(Shapes_Count_New).Height = int(df['Height'][i])
                        PPTSlide.Shapes(Shapes_Count_New).Left = int(df['Left'][i])
                        PPTSlide.Shapes(Shapes_Count_New).Top = int(df['Top'][i])

    #                     print(df['Width'][ind_chart-1],df['Height'][ind_chart-1],df['Left'][ind_chart-1],df['Top'][ind_chart-1])

                ind_chart += 1

        ind_sheet += 1

    # Save the presentation.
    SavePPT (PPTApp, PPTPres, Output_Result)              ### Save(As)
    ClosePPT (PPTApp, PPTPres, Open_presentations_list)   ### Close the presentation
    QuitApp (PPTApp, Open_presentations_list)             ### Quit the dispatched App

In [5]:
AutomatePPT()

Exporting Chart Chart 1 from Worksheet C1
Exporting Chart Chart 1 from Worksheet C2
Exporting Chart Chart 2 from Worksheet C2
Status: Closing Presentation ...
Status: Dispatched PowerPoint Object Killed.


In [ ]:
######################################### Debug ##############################

In [6]:
import win32com.client

PPTApp = win32com.client.Dispatch("PowerPoint.Application")
print("Count of Active Presentations", PPTApp.Presentations.Count)

# Names of Active Presentations
for ppts in PPTApp.Presentations:
    print(ppts.Name)

#Close all active presentations
for ppts in PPTApp.Presentations:
    PPTApp.Presentations(ppts.Name).Close()
    
# Quit dispatched Powerpoint Object
PPTApp.Quit()  

Count of Active Presentations 0


In [12]:
import win32com.client

Excel = win32com.client.Dispatch("Excel.Application")
print("Count of Active Workbooks", Excel.Workbooks.Count)

# Names of Active Presentations
for Workbook in Excel.Workbooks:
    print(Workbook.Name)

#Close all active presentations
for Workbook in Excel.Workbooks:
    Excel.Workbooks(Workbook.Name).Close()
    
# Quit dispatched Powerpoint Object
Excel.Quit()  

Count of Active Workbooks 0
